In [10]:
import numpy as np
import pandas as pd
import scipy.io as io
import os

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

torch.cuda.is_available()

True

In [12]:
import timm
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [13]:
import wandb
# WANDB_NOTEBOOK_NAME = 'Train_Analysis_VRB_wandb_sweep.ipynb'
wandb.login()

True

In [14]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': 
    {
        'batch_size': {'values': [32, 64]},
        'epochs': {'values': [5, 10, 15]},
        'lr': {'values': [0.001, 0.0005, 0.0001]},
        'model': {'values': ['efficientnet_b0','efficientnet_b2','efficientnet_b4','resnet18d','resnet34d','resnet50d','resnet101d']}
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project='VRB-Pytorch-test')


Create sweep with ID: c5w3i4pk
Sweep URL: https://wandb.ai/xiaosuhu86/VRB-Pytorch-test/sweeps/c5w3i4pk


In [15]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [16]:
class FNIRS_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.targets = self.img_labels['Label']

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 4])
        image = np.float32(io.loadmat(img_path)['fnirsimg']).reshape(3,21,45)
        label = self.img_labels.iloc[idx, 5]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()     # Optional when not using Model Specific layer

    total_correct = 0
    total_instances = 0
    y_true=[]
    y_pred=[]

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)
        classifications = torch.argmax(model(X), dim=1)
        correct_predictions = sum(classifications==y).item()

        total_correct+=correct_predictions
        total_instances+=len(y)

        y_pred.extend(classifications.data.cpu().numpy())
        y_true.extend(y.data.cpu().numpy())
            
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"val_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    acc = total_correct/total_instances

    # constant for classes
    classes = ('No_Pain','Pain')
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
    
    TP = df_cm['Pain']['Pain']
    TN = df_cm['No_Pain']['No_Pain']

    print(f"val_acc: {acc:>7f} TP: {TP:>4f} TN: {TN:>4f}")

    return loss, acc, TP, TN

In [18]:
# Load the data with imbalanced weights
train_data=FNIRS_Dataset(
    '../Label_VRB.csv',
    '../VRBdata/'
)

total_targets=torch.asarray(train_data.targets)
train_idx, valid_idx= train_test_split(
    np.arange(len(total_targets)), test_size=0.2, random_state=42, shuffle=True, stratify=total_targets)

train_set = torch.utils.data.Subset(train_data, train_idx)
# val_set = torch.utils.data.Subset(train_data, valid_idx)

train_sample_count = torch.tensor(
    [(total_targets[train_idx] == t).sum() for t in torch.unique(total_targets, sorted=True)])
train_weight = 1. / train_sample_count.float()
train_sample_weight = torch.tensor([train_weight[t] for t in total_targets[train_idx]])

#Creating PT data samplers
train_sampler = WeightedRandomSampler(train_sample_weight, len(train_sample_weight))
valid_sampler = SubsetRandomSampler(valid_idx)

def main():
    run = wandb.init()

    batch_size = wandb.config.batch_size

    # Create data loaders:
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                                    sampler=valid_sampler)

    # Model defining
    model = timm.create_model(wandb.config.model, num_classes=2, pretrained=False)
    model=model.to(device)
    
    # Other learning parameters
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
    epochs = wandb.config.epochs

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc, TP, TN = validate(validation_loader, model, loss_fn)

        wandb.log({
        'epoch': t+1, 
        #'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss,
        'true_pos': TP,
        'true_neg': TN,
        'model': wandb.config.model
        })

wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: 10bqsfkw with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 4.002099  [   32/11536]
loss: 0.840533  [ 3232/11536]
loss: 0.759519  [ 6432/11536]
loss: 0.925538  [ 9632/11536]
val_loss: 0.753798  [   32/14420]
val_acc: 0.345700 TP: 0.756447 TN: 0.289152
Epoch 2
-------------------------------
loss: 0.961215  [   32/11536]
loss: 0.732670  [ 3232/11536]
loss: 0.675555  [ 6432/11536]
loss: 0.616873  [ 9632/11536]
val_loss: 0.356713  [   32/14420]
val_acc: 0.751734 TP: 0.234957 TN: 0.822880
Epoch 3
-------------------------------
loss: 0.448856  [   32/11536]
loss: 0.560143  [ 3232/11536]
loss: 0.537909  [ 6432/11536]
loss: 0.612081  [ 9632/11536]
val_loss: 0.523153  [   32/14420]
val_acc: 0.682732 TP: 0.355301 TN: 0.727811
Epoch 4
-------------------------------
loss: 0.500281  [   32/11536]
loss: 0.562924  [ 3232/11536]
loss: 0.529392  [ 6432/11536]
loss: 0.312248  [ 9632/11536]
val_loss: 1.581232  [   32/14420]
val_acc: 0.741331 TP: 0.255014 TN: 0.808284
Epoch 5
-------------------------------
loss: 0.

epoch,▁▃▅▆█
train_loss,▆▆██▁
true_neg,▁▇▆▇█
true_pos,█▂▃▂▁
val_acc,▁▇▆▇█
val_loss,▂▁█▂▁
epoch,5
model,efficientnet_b0
train_loss,0.12997
true_neg,0.89073
true_pos,0.16332


wandb: Agent Starting Run: 3fpadx09 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 1.070545  [   32/11536]
loss: 1.637313  [ 3232/11536]
loss: 1.087544  [ 6432/11536]
loss: 0.648832  [ 9632/11536]
val_loss: 1.193606  [   32/14420]
val_acc: 0.745839 TP: 0.237822 TN: 0.815779
Epoch 2
-------------------------------
loss: 0.809367  [   32/11536]
loss: 0.768766  [ 3232/11536]
loss: 0.609572  [ 6432/11536]
loss: 0.642305  [ 9632/11536]
val_loss: 1.423412  [   32/14420]
val_acc: 0.655340 TP: 0.369628 TN: 0.694675
Epoch 3
-------------------------------
loss: 0.544616  [   32/11536]
loss: 0.523527  [ 3232/11536]
loss: 0.662273  [ 6432/11536]
loss: 0.384791  [ 9632/11536]
val_loss: 0.399445  [   32/14420]
val_acc: 0.736824 TP: 0.266476 TN: 0.801578
Epoch 4
-------------------------------
loss: 0.502466  [   32/11536]
loss: 0.417724  [ 3232/11536]
loss: 0.349220  [ 6432/11536]
loss: 0.300334  [ 9632/11536]
val_loss: 0.743630  [   32/14420]
val_acc: 0.711859 TP: 0.286533 TN: 0.770414
Epoch 5
-------------------------------
loss: 0.

...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▂▂▃▁▁▃▂▁▁▁▃▁▁▁
true_neg,▄▄▁▅▆▆▇▄▆▅▇▆▅▆█
true_pos,▄▅█▄▃▄▃▅▅▄▂▃▄▃▁
val_acc,▄▄▁▅▆▆▇▄▆▅▇▆▅▆█
val_loss,▃▁▃▂▂▂▆▁▁▁▅▃█▇▁
epoch,15
model,resnet50d
train_loss,0.00461
true_neg,0.98264
true_pos,0.0659


wandb: Agent Starting Run: q30ya2gg with config:
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	lr: 0.0001
wandb: 	model: resnet101d
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 0.830373  [   64/11536]
loss: 0.545885  [ 6464/11536]
val_loss: 0.641853  [   64/14420]
val_acc: 0.650139 TP: 0.481375 TN: 0.673373
Epoch 2
-------------------------------
loss: 0.473406  [   64/11536]
loss: 0.167871  [ 6464/11536]
val_loss: 0.630132  [   64/14420]
val_acc: 0.750000 TP: 0.323782 TN: 0.808679
Epoch 3
-------------------------------
loss: 0.091241  [   64/11536]
loss: 0.105137  [ 6464/11536]
val_loss: 0.704431  [   64/14420]
val_acc: 0.857836 TP: 0.071633 TN: 0.966075
Epoch 4
-------------------------------
loss: 0.071331  [   64/11536]
loss: 0.035259  [ 6464/11536]
val_loss: 0.398946  [   64/14420]
val_acc: 0.847781 TP: 0.148997 TN: 0.943984
Epoch 5
-------------------------------
loss: 0.011184  [   64/11536]
loss: 0.014504  [ 6464/11536]
val_loss: 0.826903  [   64/14420]
val_acc: 0.739251 TP: 0.326648 TN: 0.796055
Epoch 6
-------------------------------
loss: 0.019488  [   64/11536]
loss: 0.023240  [ 6464/11536]
val_loss: 

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▂▂▂▁▂▁▁▁▁▁▄▁▃▁
true_neg,▁▄█▇▄▇▅▇█▇▇▇███
true_pos,█▅▁▃▅▃▅▂▁▃▃▂▁▁▁
val_acc,▁▄█▇▄▇▅▇██▇▇███
val_loss,▃▁▄█▃▃▁█▁▅▁▁▁▁▆
epoch,15
model,resnet101d
train_loss,0.00343
true_neg,0.96292
true_pos,0.08596


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
